In [1]:
""" Import required packages """

import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import LoadSaveFunctions as lsf

In [ ]:
""" Load netCDF File and extract variables"""

file_path = lsf.load_file()
dataset = nc.Dataset(file_path)

lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
depth = dataset.variables['depth'][:]
d18Osw = dataset.variables['d18o'][:]

dataset.close()

print('Min value: {:.2f}; Max value: {:.2f}'.format(d18Osw.min(), d18Osw.max()))

In [ ]:
# Coordinates to search for
target_latitude = 40.0   # Replace with your target latitude
target_longitude = -20.0  # Replace with your target longitude
target_depth = 0.0      # Replace with your target depth

# Find the indices of the nearest latitude, longitude, and depth
lat_idx = np.abs(lat - target_latitude).argmin()
lon_idx = np.abs(lon - target_longitude).argmin()
depth_idx = np.abs(depth - target_depth).argmin()

# Get the oxygen isotope value at the nearest coordinates
d18Osw_value = d18Osw[depth_idx, lat_idx, lon_idx]

# Output the result
print('Oxygen isotope composition of seawater: {:.2f} ‰ (V-SMOW)'.format(d18Osw_value))

In [ ]:
"""Plot map at given depth"""

# Function to find the nearest depth value
def find_nearest_depth(depth_array, input_depth):
    idx = (np.abs(depth_array - input_depth)).argmin()
    return idx

# Choose depth
dpth = 20 #insert positive value in meters
nearest_depth_idx = find_nearest_depth(depth, dpth)

fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

# Grid and axis format
gl = ax.gridlines(draw_labels=True, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

# Retrieve data at given depth
data_at_depth = d18Osw[nearest_depth_idx, :, :]

# trace map
contour = ax.contourf(lon, lat, data_at_depth, 40, transform=ccrs.PlateCarree(), cmap='coolwarm')
cbar = plt.colorbar(contour, ax=ax, orientation='vertical', label='δ¹⁸Oₛₑₐᵥᵥₐₜₑᵣ (‰ V-SMOW)')
ax.set_title(f'seawater oxygen isotope composition at {depth[nearest_depth_idx]} meters depth')

# plt.show()